# Oanda Demo Trading Notebook

## Packages & Tools

In [1]:
from utils.packages import *
from utils.tools import *

## Tools

In [2]:
'''def make_order(accountID, stop_price, instrument, units, order_type):
    stopLossOnFill = StopLossDetails(price=stop_price)
    
    if order_type == 'long':
        units = units * 1
    elif order_type == 'short':
        units = units * (-1)

    ordr = MarketOrderRequest(
        instrument = instrument,
        units=units,
        stopLossOnFill=stopLossOnFill.data)

    r = orders.OrderCreate(accountID, data=ordr.data)
    rv = api.request(r)
    return(rv)'''

"def make_order(accountID, stop_price, instrument, units, order_type):\n    stopLossOnFill = StopLossDetails(price=stop_price)\n    \n    if order_type == 'long':\n        units = units * 1\n    elif order_type == 'short':\n        units = units * (-1)\n\n    ordr = MarketOrderRequest(\n        instrument = instrument,\n        units=units,\n        stopLossOnFill=stopLossOnFill.data)\n\n    r = orders.OrderCreate(accountID, data=ordr.data)\n    rv = api.request(r)\n    return(rv)"

In [3]:
'''def close_order(accountID, order_type, instrument):
    data_long = {"longUnits": "ALL"}
    data_short = {"shortUnits": "ALL"}
    
    if order_type == 'long':
        data = data_long
    elif order_type == 'short':
        data = data_short
        
    r = positions.PositionClose(accountID=accountID,
                                instrument=instrument,
                                data=data)
    rv = api.request(r)
    return(rv)'''

'def close_order(accountID, order_type, instrument):\n    data_long = {"longUnits": "ALL"}\n    data_short = {"shortUnits": "ALL"}\n    \n    if order_type == \'long\':\n        data = data_long\n    elif order_type == \'short\':\n        data = data_short\n        \n    r = positions.PositionClose(accountID=accountID,\n                                instrument=instrument,\n                                data=data)\n    rv = api.request(r)\n    return(rv)'

In [4]:
'''def parse_output_data(trans_rv, output_data):
    time_stamp = trans_rv['transaction']['fullPrice']['timestamp']
    dt, full_time = time_stamp.split(sep = 'T')
    tm, time_fraction = full_time.split(sep = '.')
    inst = trans_rv['transaction']['instrument']
    unts = int(trans_rv['transaction']['units'])
    prfit_pips = trans_rv['transaction']['pl']
    reason = trans_rv['transaction']['reason']
    ordrID = trans_rv['transaction']['orderID']

    if unts > 0:
        ordr_type = 'short'
    elif unts < 0:
        ordr_type = 'long'

    unts = abs(unts)

    output_data['date'].append(dt)  
    output_data['time'].append(tm)
    output_data['instrument'].append(inst)
    output_data['ordr_type'].append(ordr_type)
    output_data['units'].append(unts)
    output_data['profit_pips'].append(prfit_pips)
    output_data['orderID'].append(ordrID)
    output_data['close_reason'].append(reason)
    return(output_data)'''

"def parse_output_data(trans_rv, output_data):\n    time_stamp = trans_rv['transaction']['fullPrice']['timestamp']\n    dt, full_time = time_stamp.split(sep = 'T')\n    tm, time_fraction = full_time.split(sep = '.')\n    inst = trans_rv['transaction']['instrument']\n    unts = int(trans_rv['transaction']['units'])\n    prfit_pips = trans_rv['transaction']['pl']\n    reason = trans_rv['transaction']['reason']\n    ordrID = trans_rv['transaction']['orderID']\n\n    if unts > 0:\n        ordr_type = 'short'\n    elif unts < 0:\n        ordr_type = 'long'\n\n    unts = abs(unts)\n\n    output_data['date'].append(dt)  \n    output_data['time'].append(tm)\n    output_data['instrument'].append(inst)\n    output_data['ordr_type'].append(ordr_type)\n    output_data['units'].append(unts)\n    output_data['profit_pips'].append(prfit_pips)\n    output_data['orderID'].append(ordrID)\n    output_data['close_reason'].append(reason)\n    return(output_data)"

In [5]:
'''def get_wma_output_data(accountID, instrument, end_reason):
    last_position_r = positions.PositionDetails(accountID, instrument)
    last_position_rv = api.request(last_position_r)
    transactionID = int(last_position_rv['lastTransactionID'])

    output_data = {'date':[], 'time':[], 'instrument':[], 'ordr_type':[], 'units':[], 'profit_pips':[], 'close_reason':[], 'orderID':[], 'end_reason':[]}
    output_data['end_reason'].append(end_reason)   
    
    params = {'instruments': instrument}
    trans_r = trans.TransactionDetails(accountID=accountID, transactionID=transactionID)
    trans_rv = api.request(trans_r)
    last_order_type = trans_rv['transaction']['type']

    if last_order_type == 'ORDER_CANCEL':
        trans_r = trans.TransactionDetails(accountID=accountID, transactionID=(transactionID-1))
        trans_rv = api.request(trans_r)
        output_data = parse_output_data(trans_rv, output_data)

    elif last_order_type == 'ORDER_FILL':
        trans_r = trans.TransactionDetails(accountID=accountID, transactionID = transactionID)
        trans_rv = api.request(trans_r)
        reason = trans_rv['transaction']['reason']
        if reason == 'STOP_LOSS_ORDER':
            output_data = parse_output_data(trans_rv, output_data)

    if len(output_data['date']) > 0:
        output_df = pd.DataFrame()
        output_df = pd.DataFrame(output_data)    
        output_df.to_csv('data/machine_use/wma_log.csv', mode='a', header=False, index = False)'''

"def get_wma_output_data(accountID, instrument, end_reason):\n    last_position_r = positions.PositionDetails(accountID, instrument)\n    last_position_rv = api.request(last_position_r)\n    transactionID = int(last_position_rv['lastTransactionID'])\n\n    output_data = {'date':[], 'time':[], 'instrument':[], 'ordr_type':[], 'units':[], 'profit_pips':[], 'close_reason':[], 'orderID':[], 'end_reason':[]}\n    output_data['end_reason'].append(end_reason)   \n    \n    params = {'instruments': instrument}\n    trans_r = trans.TransactionDetails(accountID=accountID, transactionID=transactionID)\n    trans_rv = api.request(trans_r)\n    last_order_type = trans_rv['transaction']['type']\n\n    if last_order_type == 'ORDER_CANCEL':\n        trans_r = trans.TransactionDetails(accountID=accountID, transactionID=(transactionID-1))\n        trans_rv = api.request(trans_r)\n        output_data = parse_output_data(trans_rv, output_data)\n\n    elif last_order_type == 'ORDER_FILL':\n        trans_r 

## API Setup

Read from config file

In [6]:
temp_file = 'config/access_keys.yaml'
with open(temp_file) as temp_file:
    config = yaml.load(temp_file)
    
temp_file = 'config/currencies.yaml'
with open(temp_file) as temp_file:
    currencies = yaml.load(temp_file)

temp_file = 'config/stepped_loss.yaml'
with open(temp_file) as temp_file:
    loss_limits = yaml.load(temp_file)
    
temp_file = 'config/wma.yaml'
with open(temp_file) as temp_file:
    wma = yaml.load(temp_file)    

In [7]:
instrument = wma['parameter']['instrument']
short_moving_window = wma['parameter']['short_moving_window']
long_moving_window = wma['parameter']['long_moving_window']
lists_size = wma['parameter']['lists_size']

access_token = config['oanda_demo_account']['token']
accountID = config['oanda_demo_account']['account_id']
api = API(access_token = access_token)

## Code Engine

In [8]:
loss_limit_num = 3

pip = 0.0001
loss_limit = loss_limit_num * pip

In [9]:
params = {'instruments': instrument}
r = pricing.PricingStream(accountID=accountID, params=params)
rv = api.request(r)

short_wma = 0
short_flag = True
short_list = collections.deque([])
short_wma_list = collections.deque([])
short_weights = get_weights(short_moving_window)

long_wma = 0
long_flag = True
long_list = collections.deque([])
long_wma_list = collections.deque([])
long_weights = get_weights(long_moving_window)

tick_list = collections.deque([])

long_list_counter = 0
list_ready = False
old_direction =  'none'

order_flag = False

profit_target = 0.25 # pip target of actual price
price_allowed_buffer = 0.1 # percentage of SWMA decrease

profit_target = profit_target * pip 

pl_list = collections.deque([])
consecutive_losses = 2
skip_iter = 1
max_skips = 1

negative_run =  False
pl_pip = 0


valid_iteration = True

units = 1

first_position_changed = False
initial_position_captured = False

for i, resp in tqdm(enumerate(rv)):        
    resp_type = resp['type']       

    if resp_type == 'HEARTBEAT': # Heart beat response to keep the api connection alive (Avoid timeout)
        pass

    #-----------------------------------------------------------------------------------------------------
    elif resp_type == 'PRICE': # Check if we are yet to make the order               
        date_val, time_val, time_fraction = get_date_time(resp) # Get time stamp for reference            
        sell_price, buy_price, spread, tick_price = get_prices(resp) # Get prices from the response  

    #List Prep
    #-----------------------------------------------------------------------------------------------------
        #Short list prep
        #-----------------------------------------------------------------------------------------------------
        if len(short_list) < short_moving_window:
            short_list.append(tick_price)

        if short_flag and len(short_list) == short_moving_window:
            short_wma = sum(np.array(short_list)*short_weights)
            short_flag =False

        if short_flag == False and len(short_list) == short_moving_window:
            short_list.popleft()   
            short_list.append(tick_price) 
            short_wma = sum(np.array(short_list)*short_weights)



        #Long list prep   
        #-----------------------------------------------------------------------------------------------------                
        if len(long_list) < long_moving_window:
            long_list.append(tick_price)

        if long_flag and len(long_list) == long_moving_window:
            long_wma = sum(np.array(long_list)*long_weights)
            long_flag = False

        if long_flag == False and len(long_list) == long_moving_window:
            long_list.popleft()   
            long_list.append(tick_price) 
            long_wma = sum(np.array(long_list)*long_weights)   
            long_list_counter += 1
            if long_list_counter >= lists_size:
                list_ready = True
    #-----------------------------------------------------------------------------------------------------
  
        #Direction identification
        #----------------------------------------------------------------------------------------------------- 
        if valid_iteration == False and list_ready:
            #print(f'Gap: {max_gap_wma}, Threshold: {max_gap_wma_threshold}')
            pass
            
        elif valid_iteration and list_ready:
            if list_ready:
                if short_wma > long_wma:
                    current_position = 'positive'
                elif short_wma < long_wma:
                    current_position = 'negative'
                elif short_wma == long_wma:
                    current_position = 'same level'

            if list_ready and initial_position_captured == False:
                initial_position = current_position
                print(f'initial_position : {initial_position}')
                initial_position_captured =  True
                winsound.PlaySound('C:\\Windows\\Media\\tada.wav', winsound.SND_ASYNC) 

            if list_ready and initial_position_captured:
                if initial_position == current_position:
                    pass

                elif current_position == 'same level':
                    print('Same level, wait for next iteration')

                elif initial_position != current_position and current_position != 'same level':
                    print(f'initial_position:{initial_position}')
                    print(f'current_position:{current_position}')

                    #Select order type -------------------------------------------------------------------------
                    if current_position == 'positive':
                        old_order_type = 'short'
                        order_type = 'long'
                        stop_price = buy_price - loss_limit                

                    elif current_position == 'negative':
                        old_order_type = 'long'
                        order_type = 'short'
                        stop_price = sell_price + loss_limit                

                # Ordering -------------------------------------------------------------------------    
                #-------------------------------------------------------------------------------------------                
                    if order_flag:
                        print(f'Closing {old_order_type} positions if any')
                        close_order_log = close_order(accountID, old_order_type ,instrument, api)
                        get_wma_output_data(accountID, instrument, 'direction_change', api)
                        close_dict_key= list(close_order_log.keys())[1]
                        pl_pip = float(close_order_log[close_dict_key]['pl'])
                        order_flag = False

                        if len(pl_list) < consecutive_losses:
                            pl_list.append(pl_pip)
                        elif len(pl_list) == consecutive_losses:
                            pl_list.popleft()
                            pl_list.append(pl_pip)

                        negative_run = all(itrs < 0 for itrs in pl_list)

                    if negative_run == False:
                        print(f'Going {order_type}')
                        make_order_log = make_order(accountID, stop_price, instrument, units, order_type, api)
                        print('ordered')
                        first_long =  True
                        order_flag = True
                    else:
                        if max_skips > skip_iter:
                            print(f'Skiped iteration:{skip_iter}, pl_list : {pl_list}')
                            skip_iter += 1
                        elif max_skips <= skip_iter:
                            print(f'Skiped iteration:{skip_iter}, pl_list : {pl_list}')
                            negative_run = False
                            pl_list = collections.deque([])

                    initial_position = current_position

                    
                # Take Profit -------------------------------------------------------------------------    
                #-------------------------------------------------------------------------------------------                
                if initial_position == current_position and current_position != 'same level' and order_flag:
                    
                    if order_type == 'long':     
                        ordered_buy_price = float(make_order_log['orderFillTransaction']['fullPrice']['asks'][0]['price'])
                        profit = sell_price - ordered_buy_price 
                        
                        if first_long:
                            max_price = short_wma
                            first_long = False

                        max_price = max(short_wma, max_price)
                        buffered_max_price = max_price - (max_price * pip * price_allowed_buffer)

                        if profit >= profit_target and short_wma <= buffered_max_price:    
                            close_order_log = close_order(accountID, order_type ,instrument, api)    
                            get_wma_output_data(accountID, instrument, 'Take_profit', api)
                            order_flag = False
                            max_gap_wma = 0

                            
                    if order_type == 'short':     
                        ordered_sell_price = float(make_order_log['orderFillTransaction']['fullPrice']['bids'][0]['price'])
                        profit = ordered_sell_price - buy_price                
                        
                        if first_long:
                            min_price = short_wma
                            first_long = False

                        min_price = min(short_wma, min_price)
                        buffered_min_price = min_price + (min_price * pip * price_allowed_buffer)

                        if profit >= profit_target and short_wma >= buffered_min_price:    
                            close_order_log = close_order(accountID, order_type ,instrument, api)     
                            get_wma_output_data(accountID, instrument, 'Take_profit', api)
                            order_flag = False
                            max_gap_wma = 0                  

118it [01:43,  1.87s/it]

initial_position : negative


136it [02:00,  1.32s/it]

initial_position:negative
current_position:positive
Going long


137it [02:02,  1.30s/it]

...

ordered


296it [03:34,  1.55s/it]

initial_position:negative
current_position:positive
Closing short positions if any


296it [03:37,  1.36it/s]


KeyboardInterrupt: 